In [ ]:
import time
import requests
from pprint import pprint
import os
import json
import configparser
from tqdm import tqdm

config = configparser.ConfigParser()
config.read("settings.ini")

TOKEN_VK = config['vk_api']['access_token']
TOKEN_YADISK = input('Введите токен с Яндекс Полигона:')

class VkSaver:

    def __init__(self, user_id, token: str):
        self.user_id = user_id
        self.token = token
        self.api_version = config['vk_api']['api_version']
        self.get_photos_url = config['vk_api']['get_photos_url']
        self.download_file_path = config['files_path']['download_file_path']
        self.get_upload_url_api = config['yadisk_api']['get_upload_url_api']
        self.file_path = config['files_path']['download_file_path']
        self.mkdir_url = config['yadisk_api']['mkdir_url']
        self.json_path = config['files_path']['json_path']

    def _get_photos_from_folder(self) -> list:
        file_list = os.listdir(self.file_path)
        return file_list

    def get_photos(self, user_id):
        user_id = int(input('Введите Id пользователя:'))
        params = {'owner_id': user_id,
                  'album_id': 'profile',
                  'extended': 1,
                  'rev': 1,
                  'photo_sizes': 1,
                  'count': 5,
                  'access_token': self.token,
                  'v':'5.131'}
        response = requests.get(self.get_photos_url, params=params)
        profile_list = response.json()

        for file in tqdm(profile_list['response']['items']):
            time.sleep(1)
            self.size = file['sizes'][-1]['type']
            photo_url = file['sizes'][-1]['url']
            file_name = file['likes']['count']
            download_photo = requests.get(photo_url)
            with open(f'{self.download_file_path}/{file_name}.jpg', 'wb') as f:
                f.write(download_photo.content)

    def create_folder(self):
        params = {'path': self.file_path}
        headers = {'Content-Type': 'application/json',
                   'Authorization': TOKEN_YADISK}
        create_dir = requests.api.put(self.mkdir_url, headers=headers, params=params)

    def upload_photo(self):
        headers = {'Content-Type': 'application/json',
                   'Authorization': TOKEN_YADISK}
        logs_list = []
        for photo in tqdm(self._get_photos_from_folder()):
            time.sleep(1)
            params = {'path': f'{self.file_path}/{photo}'}
            try:
                get_upload_url = requests.get(self.get_upload_url_api, headers=headers, params=params)
                get_url = get_upload_url.json()
                upload_url = get_url['href']
                file_upload = requests.api.put(upload_url, data=open(f'{self.file_path}/{photo}', 'rb'), headers=headers)
                status = file_upload.status_code
                download_log = {'file_name': photo, 'size': self.size}
                logs_list.append(download_log)
            except Exception as err:
                print('Фотография уже загружена')

        with open(self.json_path, 'a') as file:
            json.dump(logs_list, file, indent=2)

        if 500 > status != 400:
            print('Фотографии успешно загружены!')
        else:
            print('Ошибка при загрузке фотографий')
    
if __name__ == '__main__':
    user = VkSaver(51620911, TOKEN_VK)
    save_photos = user.get_photos(user.user_id)
    user.create_folder()
    back_up = user.upload_photo()

In [3]:
import session_info
session_info.show()